In [1]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

Using TensorFlow backend.


In [2]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes, finalAct="softmax"):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
            
        # CONV => RELU => POOL
        model.add(Conv2D(32, (3, 3), padding="same",
            input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.25))
        
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        # use a *softmax* activation for single-label classification
        # and *sigmoid* activation for multi-label classification
        model.add(Dense(classes))
        model.add(Activation(finalAct))

        # return the constructed network architecture
        return model

In [3]:
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [4]:
import numpy as np
data = np.load('AMPs-ML.npz')
X = data['X']
y = data['y']
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.33, random_state=42)

INIT_LR = 1e-3
EPOCHS = 75
BS = 32
model = SmallerVGGNet.build(width=20, height=20, depth=3, classes=6, finalAct="sigmoid")

# initiallize the optimizer
opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)

# compile the model using binary cross-entropy rather than
# categorical cross-entropy -- this may seem counterintuitive for
# multi-label classification, but keep in mind that the goal here
# is to treat each output label as an independent Bernoulli distribution
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the network
print("[INFO] training network...")
model.fit(X_train, y_train,
          epochs=20,
          batch_size=64)
score = model.evaluate(X_test, y_test, batch_size=64)

[INFO] training network...
Epoch 1/20
923/923 [==============================] - 4s 4ms/step - loss: 0.9510 - acc: 0.5244
Epoch 2/20
923/923 [==============================] - 2s 2ms/step - loss: 0.8356 - acc: 0.5702
Epoch 3/20
923/923 [==============================] - 2s 2ms/step - loss: 0.7007 - acc: 0.6405
Epoch 4/20
923/923 [==============================] - 2s 2ms/step - loss: 0.6212 - acc: 0.6999
Epoch 5/20
923/923 [==============================] - 2s 2ms/step - loss: 0.5581 - acc: 0.7436
Epoch 6/20
923/923 [==============================] - 2s 2ms/step - loss: 0.4826 - acc: 0.7943
Epoch 7/20
923/923 [==============================] - 2s 2ms/step - loss: 0.4482 - acc: 0.8256
Epoch 8/20
923/923 [==============================] - 2s 2ms/step - loss: 0.4055 - acc: 0.8480
Epoch 9/20
923/923 [==============================] - 2s 2ms/step - loss: 0.3925 - acc: 0.8496
Epoch 10/20
923/923 [==============================] - 2s 2ms/step - loss: 0.3797 - acc: 0.8590
Epoch 11/20
923/923 [=

In [7]:
p = model.predict(X_test)

array([[0.90984166, 0.07327306, 0.43991274, 0.04979087, 0.06940976,
        0.05071924],
       [0.95655274, 0.02038468, 0.44112197, 0.02335489, 0.08133035,
        0.06741549],
       [0.8656207 , 0.01195062, 0.45519215, 0.0275079 , 0.04597356,
        0.0569214 ],
       ...,
       [0.85992956, 0.01210507, 0.35384986, 0.02816354, 0.03227266,
        0.0533173 ],
       [0.9709084 , 0.03101493, 0.48364452, 0.03557743, 0.08761466,
        0.06558961],
       [0.96638906, 0.19171104, 0.5074239 , 0.04765516, 0.09148818,
        0.06707128]], dtype=float32)

In [9]:
y_test

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 1.],
       [1., 0., 0., 1., 0., 1.],
       [1., 0., 1., 0., 0., 0.]])